# Python OOP Fundamentals

This notebook teaches Object-Oriented Programming (OOP) basics in Python by **building reusable classes** step-by-step.

By the end, we'll have a working mini card game using these classes:
- `Card`
- `Deck`
- `Hand`
- `Player`
- `Game`

Along the way we’ll learn:
- classes and objects
- `__init__` and attributes
- methods
- encapsulation (keeping logic inside the right class)
- composition (objects containing other objects)
- simple inheritance

## Why OOP?

OOP helps us organize code by bundling:
- **data** (attributes / state)
- **behavior** (methods)

Instead of scattered variables and functions, we model real concepts:
- A `Card` has a `rank` and `suit`
- A `Deck` *contains* many `Card`s and can `shuffle()` and `draw()`
- A `Player` has a `Hand` and can `play_card()`


In [ ]:
class Greeter:
    def say_hello(self):
        print("Hello!")

g = Greeter()
g.say_hello()

## Questions
- What is the name of the class/object/method?
- How many Greeter objects exist?
- If g1.say_hello() is called, does it affect g2?
- Add a new method say_goodbye() to Greeter.

## Class vs Object

- A **class** is a blueprint.
- An **object** is an instance of that blueprint.
- A **method** is a function that belongs to a class.

Example:
- `Greeter` is the class
- `g` is an object created from that class
- `say_hello` is a method in the `Greeter` class

## `__init__`, `self`, and attributes

- `__init__` runs when you create a new object.
- `self` refers to "this specific object."
- Attributes belong to the object (each object gets its own copy).


In [ ]:
class Card:
    def __init__(self, rank, suit):
        self.rank = rank # attribute
        self.suit = suit # attribute

    def describe(self):
        return f"{self.rank} of {self.suit}"

c1 = Card("A", "Spades")
c2 = Card("10", "Hearts")

print(c1.describe())
print(c2.describe())


## Questions

```python
c = Card("Q", "Hearts")
```

c.rank = ?

c.suit = ?

**Where do these values live? Are they part of the class or the object?**

In the below code:

- What error will happen?
- Why?

In [ ]:
class Dog:
    def __init__(breed, age):
        breed = breed
        age = age


## Encapsulation

Instead of writing logic *outside* the object…

Bad style:
```python
# scattered logic
if card.rank == "A": ...
```

Better OOP style: put logic inside the class and make the class responsible for its own behavior

Next: Teach `Card` how to compute its own value.

In [ ]:
class Card:
    RANK_VALUES = {
        "2": 2, "3": 3, "4": 4, "5": 5, "6": 6,
        "7": 7, "8": 8, "9": 9, "10": 10,
        "J": 11, "Q": 12, "K": 13, "A": 14
    }

    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def describe(self):
        return f"{self.rank} of {self.suit}"

    def value(self):
        return Card.RANK_VALUES[self.rank]

c = Card("K", "Diamonds")
print(c.describe(), "=>", c.value())


In [ ]:
c1 = Card("K", "Diamonds")
c2 = Card("A", "Spades")

c2.value() > c1.value()

## Composition (building bigger systems from smaller classes)

Now we’ll make a `Deck` that contains many `Card` objects.

- a `Deck` is not just data; it has behavior (`shuffle`, `draw`)
- it manages its internal cards

In [ ]:
import random

class Deck:
    SUITS = ["Hearts", "Diamonds", "Clubs", "Spades"]
    RANKS = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]

    def __init__(self):
        self._cards = [] # underscore hints: "internal use"
        self.reset()

    def reset(self):
        self._cards = [Card(rank, suit) for suit in Deck.SUITS for rank in Deck.RANKS]

    def shuffle(self):
        random.shuffle(self._cards)

    def draw(self):
        if not self._cards:
            return None
        return self._cards.pop()

    def cards_left(self):
        return len(self._cards)


deck = Deck()
deck.shuffle()
print("Cards left:", deck.cards_left())
print("Drew:", deck.draw().describe())
print("Cards left:", deck.cards_left())


## Questions

How many Card objects are created when a Deck is created?

## Hand

A `Hand` represents the cards a player holds.
It should handle things like:
- adding cards
- playing a card
- showing contents
- computing strongest card, etc.

Notice how we reuse `Card` again.


In [ ]:
class Hand:
    def __init__(self):
        self._cards = []

    def add(self, card):
        if card is not None:
            self._cards.append(card)

    def size(self):
        return len(self._cards)

    def show(self):
        """Return pretty strings for display."""
        return [c.describe() for c in self._cards]

    def get_cards(self):
        """Expose cards as a read-only copy for decision making."""
        return list(self._cards)

    def play_index(self, idx):
        """Remove and return the card at position idx."""
        if idx < 0 or idx >= len(self._cards):
            return None
        return self._cards.pop(idx)


h = Hand()
h.add(Card("A", "Spades"))
h.add(Card("5", "Hearts"))
print("Hand:", h.show())
print(h.get_cards())
print("Played:", h.play_index(0).describe())
print("Hand now:", h.show())

## Player

A `Player` object should store:
- name
- a `Hand`
- score (for games that score points)

This shows a common OOP structure:
- `Player` *contains* a `Hand`


In [ ]:
class Player:
    def __init__(self, name):
        self.name = name
        self.hand = Hand()
        self.score = 0

    def draw_from(self, deck, n=1):
        for _ in range(n):
            self.hand.add(deck.draw())

    def play_card(self):
        return self.hand.play()

    def add_point(self):
        self.score += 1


p = Player("Alice")
deck = Deck()
deck.shuffle()
p.draw_from(deck, n=3)
print(p.name, "has:", p.hand.show())


## Questions

Which objects exist inside a Player?

# Capstone: Human vs Computer

- You and the computer each start with a hand of cards (e.g., 7 cards).
- Each round, you both **choose** a card from your hand to play.
- Higher value wins the round and earns **1 point**.
- After all rounds, highest score wins.

**Player agency:**
- You must decide when to spend high cards.
- You can try to bait the computer into wasting good cards.
- You can save strong cards for later rounds.

**Computer strategy:**
- It plays the **lowest card that still beats** your chosen card.
- If it cannot beat your card, it plays its **lowest** card (saving better ones).

Notice we need all our previous classes!
- `Card` provides `value()`
- `Deck` handles creation + shuffling + dealing
- `Hand` stores and removes chosen cards
- `Player` owns a `Hand`
- `Game` manages rules and flow


In [ ]:
class HumanPlayer(Player):
    def choose_card_index(self):
        # Print indexed hand for user selection
        print(f"\n{self.name}'s hand:")
        cards = self.hand.get_cards()
        for i, c in enumerate(cards):
            print(f"  [{i}] {c.describe()} (value {c.value()})")

        # Basic input loop
        while True:
            raw = input("Choose a card index to play: ")
            if raw.isdigit():
                idx = int(raw)
                if 0 <= idx < self.hand.size():
                    return idx
            print("Invalid choice. Try again.")


class ComputerPlayer(Player):
    def choose_card_index(self, opponent_card):
        """
        Strategy:
        - Find all cards that beat opponent_card.
        - If any, play the one with the smallest value that still wins.
        - Otherwise, play the lowest-value card.
        """
        cards = self.hand.get_cards()
        beaters = [(i, c) for i, c in enumerate(cards) if c.value() > opponent_card.value()]

        if beaters:
            # play minimum winning card
            best_i, best_card = min(beaters, key=lambda x: x[1].value())
            return best_i

        # can't win -> throw lowest card
        lowest_i, lowest_card = min(enumerate(cards), key=lambda x: x[1].value())
        return lowest_i


In [ ]:
class SmartHighCardGame:
    def __init__(self, human_name="You", hand_size=7):
        self.deck = Deck()
        self.deck.shuffle()

        self.human = HumanPlayer(human_name)
        self.cpu = ComputerPlayer("Computer")

        self.hand_size = hand_size
        self._deal_hands()

    def _deal_hands(self):
        self.human.draw_from(self.deck, self.hand_size)
        self.cpu.draw_from(self.deck, self.hand_size)

    def _round_winner(self, human_card, cpu_card):
        if human_card.value() > cpu_card.value():
            return self.human
        elif cpu_card.value() > human_card.value():
            return self.cpu
        return None

    def play(self):
        print("=== Smart High Card: You vs Computer ===")
        rounds = min(self.human.hand.size(), self.cpu.hand.size())

        for r in range(1, rounds + 1):
            print(f"\n--- Round {r}/{rounds} ---")
            print(f"Score: {self.human.name} {self.human.score} - {self.cpu.score} {self.cpu.name}")

            # Human chooses first (agency)
            human_idx = self.human.choose_card_index()
            human_card = self.human.hand.play_index(human_idx)

            # CPU responds with strategy
            cpu_idx = self.cpu.choose_card_index(human_card)
            cpu_card = self.cpu.hand.play_index(cpu_idx)

            print(f"\n{self.human.name} played: {human_card.describe()} ({human_card.value()})")
            print(f"{self.cpu.name} played: {cpu_card.describe()} ({cpu_card.value()})")

            winner = self._round_winner(human_card, cpu_card)
            if winner is None:
                print("Result: Tie! No points.")
            else:
                winner.add_point()
                print(f"Result: {winner.name} wins the round!")

        print("\n=== Final Score ===")
        print(f"{self.human.name}: {self.human.score}")
        print(f"{self.cpu.name}: {self.cpu.score}")

        if self.human.score > self.cpu.score:
            print("Overall Winner:", self.human.name)
        elif self.cpu.score > self.human.score:
            print("Overall Winner:", self.cpu.name)
        else:
            print("Overall Result: Tie!")

game = SmartHighCardGame(human_name="R R", hand_size=7)
game.play()
